# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json
import gmaps.datasets
# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityweather=pd.read_csv("../python-api-challenge/cityweather.csv")
cityweather


,City,longitud,Latitud,Maximum Temperture,Humidity,Cloudines,Wind Speed,Country
0,cidreira,-50.21,-30.18,63.63,85,97,15.30,BR
1,cap malheureux,57.61,-19.98,75.00,70,67,13.00,MU
2,rikitea,-134.97,-23.12,74.30,73,100,9.55,PF
3,saskylakh,114.08,71.92,-1.97,87,35,2.28,RU
4,busselton,115.33,-33.65,68.00,26,0,10.51,AU
...,...,...,...,...,...,...,...,...
535,camiri,-63.52,-20.05,69.80,73,90,5.82,BO
536,baker city,-117.83,44.77,41.00,79,1,2.26,US
537,uong bi,106.78,21.03,75.20,73,90,11.41,VN
538,san antonio del norte,-87.70,13.88,69.80,88,40,5.91,HN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
places=cityweather[["Latitud","longitud"]]

In [5]:
humidity=cityweather["Humidity"]

In [6]:
fig = gmaps.figure(center=(40.0, 5.0), zoom_level=1)
max_intensity = np.max(humidity)
Hlayer=gmaps.heatmap_layer(places,weights=humidity,max_intensity=100, point_radius=5)
fig.add_layer(Hlayer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
selectcity_df = cityweather.loc[(cityweather["Maximum Temperture"] >= 70) & (cityweather["Maximum Temperture"] <= 80) & (cityweather["Wind Speed"] <= 10) & (cityweather["Cloudines"] == 0)].dropna()
selectcity_df

,City,longitud,Latitud,Maximum Temperture,Humidity,Cloudines,Wind Speed,Country
120,tezu,96.17,27.92,74.91,58,0,3.69,IN
261,gondar,37.47,12.60,74.70,31,0,3.78,ET
263,mackay,149.20,-21.15,77.00,83,0,8.05,AU
271,awbari,12.78,26.59,79.86,26,0,9.86,LY
298,nalut,10.85,30.33,72.36,27,0,7.36,LY
332,pasighat,95.33,28.07,73.83,66,0,3.29,IN
387,chenghai,116.77,23.46,77.00,69,0,8.95,CN
444,motril,-3.52,36.75,71.01,47,0,3.00,ES
510,yeppoon,150.73,-23.13,71.60,53,0,6.93,AU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=selectcity_df.loc[:,["City","Country","Latitud","longitud"]]

In [9]:
hotel_df['Hotel Name']= ""
hotel_df

,City,Country,Latitud,longitud,Hotel Name
120,tezu,IN,27.92,96.17,
261,gondar,ET,12.60,37.47,
263,mackay,AU,-21.15,149.20,
271,awbari,LY,26.59,12.78,
298,nalut,LY,30.33,10.85,
332,pasighat,IN,28.07,95.33,
387,chenghai,CN,23.46,116.77,
444,motril,ES,36.75,-3.52,
510,yeppoon,AU,-23.13,150.73,


In [10]:
nearhotel = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" # dictionary for iterations
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key} 

In [11]:
for index, row in hotel_df.iterrows():                            # finding locations using lat & lon data
    Latitud = row["Latitud"]
    longitud = row["longitud"]
    city = row["City"]
    params["location"] = f'{Latitud},{longitud}'                  # Poccesing for each location
    print(f'Proccesing........... City {index}: {city}.')
    Hotel = requests.get(nearhotel, params=params).json()          #requesting data
    results = Hotel['results']
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.") # processing the hotel
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']     # retrieving hotel
    except (IndexError, KeyError):                                 # just in case no findings
        print("Unknown Hotel........................")
    time.sleep(5)                                 # Wait 5 sec to avoid Error

print("------------End of Search---------------") # Finish processing

Proccesing........... City 120: tezu.
Closest hotel in tezu is Hotel Taboka.
Proccesing........... City 261: gondar.
Closest hotel in gondar is Haile Resort Gondar.
Proccesing........... City 263: mackay.
Closest hotel in mackay is Coral Cay Resort.
Proccesing........... City 271: awbari.
Unknown Hotel........................
Proccesing........... City 298: nalut.
Unknown Hotel........................
Proccesing........... City 332: pasighat.
Closest hotel in pasighat is Donyi Hango Camp Homestay And Resort.
Proccesing........... City 387: chenghai.
Closest hotel in chenghai is 澄海国瑞豪生大酒店.
Proccesing........... City 444: motril.
Closest hotel in motril is Elba Motril Hotel Beach & Business Hotel.
Proccesing........... City 510: yeppoon.
Closest hotel in yeppoon is Yeppoon Beach House.
------------End of Search---------------


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
places = hotel_df[["Latitud", "longitud"]]
places                                               #ubication parameters for markers

,Latitud,longitud
120,27.92,96.17
261,12.60,37.47
263,-21.15,149.20
271,26.59,12.78
298,30.33,10.85
332,28.07,95.33
387,23.46,116.77
444,36.75,-3.52
510,-23.13,150.73


In [13]:
marker = gmaps.marker_layer(places, info_box_content = hotel_info)

# Add the layer to the map, Display Map
fig.add_layer(marker)
fig


Figure(layout=FigureLayout(height='420px'))